In [ ]:
# # mount drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # move directory
# import os
# colab_dir = "./drive/MyDrive/"
# os.chdir(colab_dir)

In [1]:
# import libraries
!pip install tabulate
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib_inline
%matplotlib inline

In [2]:
# set random seed
import random
random.seed(335)

In [3]:
# magic word
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
# for better viz
import pprint
import warnings
warnings.filterwarnings('ignore')

### reference
-------------------

- [pandas cheat sheet](https://github.com/pandas-dev/pandas/tree/master/doc/cheatsheet)
- [numpy cheat sheet(data camp)](https://www.datacamp.com/community/blog/python-numpy-cheat-sheet)
- [scikit-learn cheat sheet(data camp)](datacamp.com/community/blog/scikit-learn-cheat-sheet)

# data preparation
---------------------
The data preparation phase covers all activities to construct the final dataset (data that will be fed into the modeling tool(s)) from the initial raw data. Data preparation tasks are likely to be performed multiple times and not in any prescribed order. Tasks include table, record and attribute selection as well as transformation and cleaning of data for modeling tools.

## select data
----------

### task

Decide on the data to be used for analysis. Criteria include relevance to the data mining goals, quality and technical constraints such as limits on data volume or data types. Note that data selection covers selection of attributes (columns) as well as selection of records (rows) in a table.

### output

List the data to be included/excluded and the reasons for these decisions.

#### select data source
| # | data | included/excluded | reasons | quality | volume/data types |
|:---:|:---|:---|:---|:---|:---|
| 1 | Retail | included | it quantifies sales | bad | excel table |

#### select attributes & records

| # | data | included/excluded | reasons | quality | volume/data types |
|:---:|:---|:---|:---|:---|:---|
| 1 | Quantity | included | it quantifies sales | bad | int |
| 2 | InvoiceDate | included | can be used to see patterns| medium | date |
| 3 | UnitPrice | included | it quantifies sales | bad | double |
| 4 | Country | included | signify price for traveling goods| good | string |
| 5 | CustomerID | included | used as index, give us information that customer was the same as with other product | good | int |
| 6 | StockCode | excluded | used as index, doesn't show any information | good | string |
| 7 | InvoiceNo | excluded | used as index, doesn't show any information + missing in almost all rows | very bad | int |
| 8 | Description | included | products with similar words can be used for another customers | good | string |


## clean data
------------------

### task

Raise the data quality to the level required by the selected analysistechniques. This may involve selection of clean subsets of the data, the insertion of suitable defaults or more ambitious techniques such as the estimation of missing data by modeling.

### output

Describe what decisions and actions were taken to address the data quality problems reported during the verify data quality task of the data understanding phase. Transformations of the data for cleaning purposes and the possible impact on the analysis results should be considered.


In [5]:
path = "data/"
file_name = "Online Retail.xlsx"
sheet_name = "Online Retail"
df = pd.read_excel(path + file_name, sheet_name=sheet_name)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [6]:
# remove unnncesary columns
columns = ['StockCode', 'InvoiceNo']
df = df.drop(columns=columns)

In [11]:
del_flg1 = df['Quantity'] <= 0
def_neg = df.copy()
def_neg = def_neg[del_flg1]
def_neg["sum"] = def_neg.apply(lambda row: row['Quantity']*row['UnitPrice'], axis=1)
def_neg.groupby("Country")["sum"].sum()
#price of all returned items from country

Country
Australia              -1444.04
Austria                  -44.36
Bahrain                 -205.74
Belgium                 -285.38
Channel Islands         -364.15
Cyprus                  -644.09
Czech Republic          -119.02
Denmark                 -187.20
EIRE                  -20177.14
European Community        -8.50
Finland                 -219.34
France                -12311.21
Germany                -7168.93
Greece                   -50.00
Hong Kong              -5574.76
Israel                  -227.44
Italy                   -592.73
Japan                  -2075.75
Malta                   -220.12
Netherlands             -784.80
Norway                 -1001.98
Poland                  -121.51
Portugal               -4380.08
Saudi Arabia             -14.75
Singapore             -12158.90
Spain                  -6802.53
Sweden                 -1782.42
Switzerland             -704.55
USA                    -1849.47
United Kingdom       -815291.60
Name: sum, dtype: float64

In [12]:
countries = pd.get_dummies(def_neg["Country"], dummy_na=False)
def_neg = def_neg.drop('Country', axis = 1, errors='ignore')
def_neg['InvoiceDate'] = pd.to_datetime(def_neg['InvoiceDate'])
def_neg['time'] = def_neg['InvoiceDate'].dt.hour
# Join the encoded df
def_neg = def_neg.join(countries)
def_neg

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,sum,time,Australia,Austria,Bahrain,Belgium,Channel Islands,Cyprus,Czech Republic,Denmark,EIRE,European Community,Finland,France,Germany,Greece,Hong Kong,Israel,Italy,Japan,Malta,Netherlands,Norway,Poland,Portugal,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Kingdom
141,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,-27.50,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
154,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,-4.65,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
235,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,-19.80,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
236,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,-6.96,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
237,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,-6.96,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540449,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,-9.13,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
541541,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,-224.69,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
541715,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,-54.75,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
541716,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,-1.25,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [13]:
def_neg.to_csv(path + "negative.xlsx", index=False)

In [18]:
# clean outlier
h = df["UnitPrice"].quantile(0.99)
l = df["UnitPrice"].quantile(0.1)
df = df[(df["UnitPrice"] < h) & (df["UnitPrice"] > l)]
## clean minus data
column1 = 'Quantity'
column2 = 'UnitPrice'
del_flg1 = df[column1] <= 0
del_flg2 = df[column2] <= 0
df = df.drop(df[del_flg1].index)
df = df.drop(df[del_flg2].index)

In [19]:
# clean duplicate records
df = df.drop_duplicates()

In [20]:
df

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


## construct data
----------

### task

This task includes constructive data preparation operations such as the production of derived attributes, entire new records or transformed values for existing attributes.

### output

#### derived attributes

Derived attributes are new attributes that are constructed from one or more existing attributes in the same record. 

Examples: area = length * width

#### generated records

Describe the creation of completely new records. 

Example: create records for customers who made no purchase during the past year.There was no reason to have such records in the raw data, but for modeling purposes it might make sense to explicitly represent the fact that certain customers made zero purchases.


In [21]:
# derive attributes
df["sum"] = df.apply(lambda row: row['Quantity']*row['UnitPrice'], axis=1)
#price of all items for every item
df

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sum
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


In [23]:
df.groupby("Country")["sum"].sum()
#price of all items from country

Country
Australia               1.286781e+05
Austria                 8.287280e+03
Bahrain                 7.541400e+02
Belgium                 3.678823e+04
Brazil                  1.143600e+03
Canada                  2.731520e+03
Channel Islands         1.909390e+04
Cyprus                  1.127609e+04
Czech Republic          7.349000e+02
Denmark                 1.761728e+04
EIRE                    2.547572e+05
European Community      1.237850e+03
Finland                 1.676075e+04
France                  1.758608e+05
Germany                 1.949444e+05
Greece                  4.352450e+03
Hong Kong               9.226550e+03
Iceland                 4.069760e+03
Israel                  7.518400e+03
Italy                   1.529531e+04
Japan                   3.420038e+04
Lebanon                 1.680680e+03
Lithuania               1.661060e+03
Malta                   2.003090e+03
Netherlands             2.689500e+05
Norway                  2.989138e+04
Poland                  6.6731

In [24]:
import datetime as dt

#added time column for time of the day of the selling of product
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['time'] = df['InvoiceDate'].dt.hour
#change date to ordinal numbers
df['InvoiceDate'] = df['InvoiceDate'].map(dt.datetime.toordinal)

#hot encoding of the contries
countries = pd.get_dummies(df["Country"], dummy_na=False)
df = df.drop('Country', axis = 1, errors='ignore')
# Join the encoded df
df = df.join(countries)
df = df[df["Unspecified"]==0]
df = df.drop(["Unspecified"], axis=1)

#cleaning na values
df = df[df['CustomerID'].notna()]

In [25]:
df.to_csv(path + "input.xlsx", index=False)

In [26]:
# generate records

## integrate data
-------------

### task

These are methods whereby information is combined from multiple tables or records to create new records or values.

### output

Merging tables refers to joining together two or more tables that have different information about the same objects. 

Example: a retail chainhas one table with information about each store's general characteristics(e.g., floor space, type of mall), another table with summarized sales data (e.g., profit, percent change in sales from previous year) and another with information about the demographics of the surrounding area. Each of these tables contains one record for each store. These tables can be merged together into a new table with one record foreach store, combining fields from the source tables.

Merged data also covers aggregations. Aggregation refers to operations where new values are computed by summarizing together information from multiple records and/or tables. For example, converting a table ofcustomer purchases where there is one record for each purchase into a new table where there is one record for each customer, with fields such as number of purchases, average purchase amount, percent of orders charged to credit card, percent of items under promotion, etc.


# format data
----------------

### task

Formatting transformations refer to primarily syntactic modifications made to the data that do not change its meaning, but might be required by the modeling tool.

### output

Some tools have requirements on the order of the attributes, such as the first field being a unique identifier for each record or the last field being the outcome field the model is to predict.

It might be important to change the order of the records in the dataset. Perhaps the modeling tool requires that the records be sorted according to the value of the outcome attribute. A common situation is that the records of the dataset are initially ordered in some way but the modeling algorithm needs them to be in a fairly random order. For example, when using neural networks it is generally best for the records to be presented in a random order although some tools handle this automatically with-out explicit user intervention.

Additionally, there are purely syntactic changes made to satisfy the requirements of the specific modeling tool. 

Examples: removing commas from within text fields in comma-delimited data files, trimming all values to a maximum of 32 characters.


## note/questions
-------------

#### select data
Quantity, InvoiceDate, UnitPrice, Country are selected columns. \
StockCode, CustomerID, InvoiceNo, Description are dropped.
#### clean data
rows where Quantity and UnitPrice is negative\zero are dropped, duplicates are dropped too.
#### construct data
Constructed data is:
 - Price of all items for every item
 - Time of the day of sell
 - Price of all items for every country

I didn't generate new rows, because it will make quality of data even lower, as generated data will be random.
#### integrate data
We don't need to integrate data, because we have only 1 table.
#### format data
We don't need to sort\rename data. But InvoiceDate can be converted to ordinal number
